In [3]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)
import random

In [4]:
connections.connect("default", host="localhost", port="19530")

In [56]:
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]
schema = CollectionSchema(fields, "")
demo = Collection("demo", schema)

In [77]:
demo

<Collection>:
-------------
<name>: demo
<description>: 
<schema>: {'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 8}}]}

In [59]:
entities = [
    [i for i in range(3000)],
    [[random.random() for _ in range(8)] for _ in range(3000)],
]
insert_result = demo.insert(entities)
demo.flush()  

In [60]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
demo.create_index("embeddings", index)

Status(code=0, message=)

In [65]:
demo.load()
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}
result = demo.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["id", "embeddings"])

In [66]:
result[0].ids

[2998, 2898, 884]

In [72]:
[r.entity.get('embeddings') for r in result[0]]

[[0.5126429796218872,
  0.10488633811473846,
  0.8042959570884705,
  0.9489639401435852,
  0.23087722063064575,
  0.6608924269676208,
  0.6877580285072327,
  0.5401747226715088],
 [0.6592569947242737,
  0.13888874650001526,
  0.8460841178894043,
  0.9470040798187256,
  0.29675623774528503,
  0.5355814099311829,
  0.6274703145027161,
  0.38074740767478943],
 [0.34233683347702026,
  0.07627993077039719,
  0.5676783323287964,
  0.8684123158454895,
  0.38202518224716187,
  0.7005187273025513,
  0.7894041538238525,
  0.5391728281974792]]

In [5]:
utility.list_collections()

['demo']

In [6]:
utility.has_collection("demo")

True

In [7]:
collection = Collection("demo")
collection

<Collection>:
-------------
<name>: demo
<description>: 
<schema>: {'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 8}}]}